# Proyectos bitbucket

## 📦Obtener los datos

#### Repositorios

Consultar los repositorios

In [ ]:
%pip install python-dotenv

import requests
import pandas as pd
from dotenv import load_dotenv
import os
from IPython.display import display


# Configura tus credenciales
# Cargar las variables de entorno desde el archivo .env
load_dotenv()

username = os.getenv("BITBUCKET_USERNAME")
app_password = os.getenv("BITBUCKET_APP_PASSWORD")
workspace = os.getenv("BITBUCKET_WORKSPACE")


url_repos = f"https://api.bitbucket.org/2.0/repositories/{workspace}"

response = requests.get(url_repos, auth=(username, app_password))

if response.status_code == 200:
    repositorios = response.json()
    print(f"Repositorios en {workspace}:")
    repos_data = []
    for repo in repositorios.get("values", []):
        repos_data.append(
            {
                "Nombre": repo.get("name"),
                "URL": repo.get("links", {}).get("html", {}).get("href"),
            }
        )
    repos_df = pd.DataFrame(repos_data)
    repos_df = repos_df.sort_values(
        by="Nombre"
    )  # Ordenar por nombre del repositorio en ascendente
    # print(repos_df)
else:
    print("Error al obtener repositorios:", response.status_code, response.text)

# Almacenar los repositorios en una variable para uso posterior
repositorios = [
    repo["Nombre"]
    for repo in repos_data
    if repo["Nombre"] not in ["pgp", "Pruebas_erp", "Inventario", "b2c", "efi"]
]


Listado de repositorios

In [ ]:
repos_df = pd.DataFrame(repositorios, columns=["Repositorio"])
display(repos_df)

#### Funciones

In [ ]:
# Función para obtener todos los elementos paginados
def get_all_items(url, params=None):
    items = []
    response = requests.get(url, params=params, auth=(username, app_password))
    if response.status_code != 200:
        print("Error:", response.status_code, response.text)
        return items
    data = response.json()
    items.extend(data.get("values", []))
    return items


#### Pull requests y commits

Consultar todos los estados de PR's.

Filtrar 50 registros por páginas, 3 páginas

In [ ]:
cantidad_paginas = 4
registros_por_pagina = 50  # Creo que no soporta más de 50 registros por página

# Obtener pull requests
pull_requests = []

for repo_slug in repositorios:
    # for repo_slug in ["fip", "b2b"]:
    pr_url = f"https://api.bitbucket.org/2.0/repositories/{workspace}/{repo_slug}/pullrequests"
    for page in range(1, cantidad_paginas + 1):
        pull_requests.extend(
            get_all_items(
                pr_url,
                params={"page": page, "pagelen": registros_por_pagina, "state": "ALL"},
            )
        )
        print(f"Obteniendo pull requests, repositorio: '{repo_slug}' - página: {page}")

print(f"✅Total de pull requests: {len(pull_requests)}\n")


In [ ]:
# Obtener commits
commits = []

for repo_slug in repositorios:
    # for repo_slug in ["fip", "b2b"]:
    commits_url = (
        f"https://api.bitbucket.org/2.0/repositories/{workspace}/{repo_slug}/commits"
    )
    for page in range(1, cantidad_paginas + 1):
        commits.extend(
            get_all_items(
                commits_url,
                params={"page": page, "pagelen": registros_por_pagina},
            )
        )
        print(f"Obteniendo commits, repositorio: '{repo_slug}' - página: {page}")

print(f"✅Total de commits: {len(commits)}")

Filtrar registros de más de 120 días

In [ ]:
cantidad_dias = 120


print(f"\nTotal pull requests: {len(pull_requests)}")
fecha_limite = pd.Timestamp.now(tz="America/Bogota") - pd.Timedelta(days=cantidad_dias)
pull_requests = [
    pr for pr in pull_requests if pd.to_datetime(pr["created_on"]) >= fecha_limite
]
print(f"Registros de los últimos {cantidad_dias} días: {len(pull_requests)}\n")


print(f"\nTotal commits: {len(commits)}")
fecha_limite = pd.Timestamp.now(tz="America/Bogota") - pd.Timedelta(days=cantidad_dias)
commits = [
    commit
    for commit in commits
    if pd.to_datetime(commit["date"]) >= fecha_limite
    # and "Resolve:" in commit.get("message", "")
]
print(
    f"Registros de los últimos {cantidad_dias} días y con 'Resolve:' en el mensaje: {len(commits)}\n"
)

Limpiar **pull requests**:

- Agrega campos nuevos
- Elimina campos sin uso
- Ordena por created_on DESC

In [ ]:
for pr in pull_requests:
    if "author" in pr and "nickname" in pr["author"]:
        pr["author"] = pr["author"]["nickname"]
    if "source" in pr and "branch" in pr["source"] and "name" in pr["source"]["branch"]:
        pr["branch"] = pr["source"]["branch"]["name"]
        pr["type_branch"] = (
            pr["branch"].split("/")[0] if "/" in pr["branch"] else "unknown"
        )
    if (
        "source" in pr
        and "repository" in pr["source"]
        and "name" in pr["source"]["repository"]
    ):
        pr["repository"] = pr["source"]["repository"]["name"]
    if pr.get("merge_commit") and "hash" in pr["merge_commit"]:
        pr["merge_commit"] = pr["merge_commit"]["hash"]

    # Calculate days_open based on the PR state
    if "state" in pr and "created_on" in pr:
        created_on = pd.to_datetime(pr["created_on"])
        if pr["state"] == "OPEN":
            # For open PRs: days between created_on and now
            days_open = (
                pd.Timestamp.now(tz="America/Bogota")
                - created_on.tz_convert("America/Bogota")
            ).days
        elif pr["state"] == "MERGED" and "updated_on" in pr:
            # For merged PRs: days between created_on and updated_on
            updated_on = pd.to_datetime(pr["updated_on"])
            days_open = (
                updated_on.tz_convert("America/Bogota")
                - created_on.tz_convert("America/Bogota")
            ).days
        else:
            days_open = 0
        pr["days_open"] = days_open


df_pr = pd.DataFrame(pull_requests)

print(pull_requests[0])

# Sort by created_on in descending order to show most recent records first
df_pr = df_pr.sort_values(by="created_on", ascending=False)

# Eliminar columnas innecesarias
# df_pr = df_pr.drop(
#     columns=[
#         "type",
#         "title",
#         "description",
#         "reason",
#         "destination",
#         "summary",
#         "closed_by",
#         "links",
#         "source",
#     ]
# )

Limpiar **commits**

- Agrega campos nuevos
- Elimina campos sin uso
- Ordena por created_on DESC

In [ ]:
for cm in commits:
    # Extract author name from nested dictionary
    if "author" in cm and "user" in cm["author"] and "nickname" in cm["author"]["user"]:
        cm["author"] = cm["author"]["user"]["nickname"]
    if "repository" in cm and "name" in cm["repository"]:
        cm["repository"] = cm["repository"]["name"]


df_commits = pd.DataFrame(commits)


# Sort by date in descending order to show most recent records first
df_commits = df_commits.sort_values(by="date", ascending=False)

Imprimir **pull requests**

In [ ]:
# print(type(df_pr))
print(f"Cantidad de registros: {len(df_pr)}")
display(df_pr.head(3))
# display(df_pr)

Imprimir **commits**

In [ ]:
# print(type(df_commits))
print(f"Cantidad de registros: {len(df_commits)}")
display(df_commits.head(3))

## 📈Reportes

### 1. Actividad de commits

#### Procesar los datos

In [ ]:
# Install matplotlib if needed
%pip install matplotlib

import pandas as pd
import matplotlib.pyplot as plt

# Try to import seaborn
try:
    import seaborn as sns

    has_seaborn = True
except ImportError:
    has_seaborn = False
    print("Seaborn no instalado. Algunas visualizaciones serán simplificadas.")

# Make sure author is a string, not a dictionary
df_commits["author"] = df_commits["author"].astype(str)

# Convert date to datetime
df_commits["date"] = pd.to_datetime(df_commits["date"])

# Extract time periods
df_commits["day"] = df_commits["date"].dt.date
df_commits["week"] = df_commits["date"].dt.isocalendar().week
df_commits["month"] = df_commits["date"].dt.month
df_commits["year"] = df_commits["date"].dt.year

# Create year-month field
df_commits["year_month"] = df_commits["date"].dt.strftime("%Y-%m")

#### Análisis de actividad

### 3. Pull Requests (PR)

#### 3.1 Cantidades y tiempos

##### Procesar los datos y graficar

In [ ]:
import pandas as pd
# from datetime import datetime

# Análisis de Pull Requests
import matplotlib.pyplot as plt

# Verificar los estados disponibles en los PRs
pr_states = df_pr["state"].value_counts().reset_index()
pr_states.columns = ["Estado", "Cantidad"]

# Analizar PR por autor
pr_by_author = df_pr.groupby(["author", "state"]).size().unstack().fillna(0)
if "MERGED" in pr_by_author.columns:
    pr_by_author["total"] = pr_by_author.sum(axis=1)
    pr_by_author = pr_by_author.sort_values("total", ascending=False)
    pr_by_author = pr_by_author.drop(columns=["total"])

# Calcular tiempos promedio
df_pr["created_on"] = pd.to_datetime(df_pr["created_on"])
df_pr["updated_on"] = pd.to_datetime(df_pr["updated_on"])

# Métricas de tiempo para PRs cerrados/mergeados
merged_prs = df_pr[df_pr["state"] == "MERGED"].copy()
if not merged_prs.empty:
    merged_prs["time_to_merge_days"] = (
        merged_prs["updated_on"] - merged_prs["created_on"]
    ).dt.total_seconds() / 86400
    avg_merge_time = merged_prs["time_to_merge_days"].mean()
    median_merge_time = merged_prs["time_to_merge_days"].median()
    max_merge_time = merged_prs["time_to_merge_days"].max()

# Extraer tipos de ramas de los PRs
branch_types = df_pr["type_branch"].value_counts().reset_index()
branch_types.columns = ["Tipo", "Cantidad"]


# Visualizaciones
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 📈 1. Estado de los PRs
axes[0, 0].bar(pr_states["Estado"], pr_states["Cantidad"], color="skyblue")
axes[0, 0].set_title("Estado de Pull Requests")
axes[0, 0].set_xlabel("Estado")
axes[0, 0].set_ylabel("Cantidad")

# 📈 2. PRs por autor
pr_by_author.plot(kind="bar", stacked=True, ax=axes[0, 1])
axes[0, 1].set_title("Pull Requests por Autor")
axes[0, 1].set_xlabel("Autor")
axes[0, 1].set_ylabel("Cantidad")
axes[0, 1].legend(title="Estado")


# 📈 3. Tiempo de resolución de PRs
if not merged_prs.empty:
    merged_prs.boxplot(column="time_to_merge_days", ax=axes[1, 0])
    axes[1, 0].set_title("Tiempo para Merge (días)")
    axes[1, 0].set_ylabel("Días")

    # Añadir una línea para el promedio
    axes[1, 0].axhline(
        y=avg_merge_time,
        color="red",
        linestyle="-",
        label=f"Promedio: {avg_merge_time:.1f} días",
    )
    axes[1, 0].legend()

# 📈 4. Tipos de ramas
axes[1, 1].pie(branch_types["Cantidad"], labels=branch_types["Tipo"], autopct="%1.1f%%")
axes[1, 1].set_title("Distribución de Tipos de Ramas")

plt.tight_layout()
plt.show()

# Métricas adicionales
print("\n=== Métricas de Pull Requests ===")
print(f"Total de PRs analizados: {len(df_pr)}")
print(f"PRs por estado: {dict(pr_states.values)}")
if not merged_prs.empty:
    print(f"\nTiempo promedio hasta merge: {avg_merge_time:.1f} días")
    print(f"Tiempo mediano hasta merge: {median_merge_time:.1f} días")
    print(f"Tiempo máximo hasta merge: {max_merge_time:.1f} días")


# -------------------------------------------------------------


# 📊 Análisis de eficiencia de PRs
if not merged_prs.empty:
    author_efficiency = (
        merged_prs.groupby("author")["time_to_merge_days"]
        .agg(["mean", "count"])
        .sort_values("mean")
    )
    author_efficiency.columns = ["Tiempo promedio (días)", "PRs mergeados"]

    # Crear tres gráficas para visualizar la eficiencia de los autores
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18, 5))

    # 📈 Gráfica 1: Tiempo promedio hasta merge
    author_efficiency["Tiempo promedio (días)"].plot(
        kind="bar",
        color="skyblue",
        ax=ax1,
        title="Tiempo promedio hasta merge por autor",
    )
    ax1.set_xlabel("Autor")
    ax1.set_ylabel("Días")
    for i, v in enumerate(author_efficiency["Tiempo promedio (días)"]):
        ax1.text(i, v + 0.1, f"{v:.1f}", ha="center")

    # 📈 Gráfica 2: Cantidad de PRs mergeados
    author_efficiency["PRs mergeados"].plot(
        kind="bar",
        color="lightgreen",
        ax=ax2,
        title="Cantidad de PRs mergeados por autor",
    )
    ax2.set_xlabel("Autor")
    ax2.set_ylabel("Cantidad")
    for i, v in enumerate(author_efficiency["PRs mergeados"]):
        ax2.text(i, v + 0.1, str(v), ha="center")

    # 📈 Gráfica 3: Eficiencia general de los autores
    scatter = ax3.scatter(
        author_efficiency["Tiempo promedio (días)"],
        author_efficiency["PRs mergeados"],
        s=100,
        alpha=0.7,
        c=author_efficiency["Tiempo promedio (días)"],
        cmap="coolwarm_r",
    )

    # Añadir etiquetas a cada punto
    for i, author in enumerate(author_efficiency.index):
        ax3.annotate(
            author,
            (
                author_efficiency["Tiempo promedio (días)"][i],
                author_efficiency["PRs mergeados"][i],
            ),
            xytext=(5, 5),
            textcoords="offset points",
        )

    ax3.set_title("Eficiencia de autores")
    ax3.set_xlabel("Tiempo promedio (días)")
    ax3.set_ylabel("PRs mergeados")
    ax3.grid(True, linestyle="--", alpha=0.7)

    # Añadir colorbar
    cbar = plt.colorbar(scatter, ax=ax3)
    cbar.set_label("Tiempo promedio (días)")

    plt.tight_layout()
    plt.show()

    print("\n=== Eficiencia de autores (tiempo promedio hasta merge) ===")
    print(author_efficiency.to_string(float_format=lambda x: f"{x:.1f}"))

#### 3.2 Comentarios

##### PRs más largos

In [ ]:
# Identificar PRs con tiempos de revisión largos y comentarios numerosos
if "comment_count" in df_pr.columns:
    high_discussion = df_pr.sort_values("comment_count", ascending=False).head(5)
    print("\n=== PRs con más discusión (mayor número de comentarios) ===")
    high_discussion["formatted_date"] = high_discussion["created_on"].dt.strftime(
        "%d-%m-%Y"
    )
    print(
        pd.DataFrame(
            {
                "Título": high_discussion["title"],
                "Autor": high_discussion["author"],
                "Comentarios": high_discussion["comment_count"],
                "Estado": high_discussion["state"],
                "Fecha": high_discussion["formatted_date"],
            }
        ).to_string()
    )


##### Procesar los datos (primeras 3 gráficas)

In [ ]:
import pandas as pd
# from datetime import timedelta

import matplotlib.pyplot as plt

# Parámetro configurable: número de días a analizar
num_days_to_analyze = 30
# Calcular fecha límite
today = pd.Timestamp.now(tz="America/Bogota")
date_limit = today - pd.Timedelta(days=num_days_to_analyze)

# Asegurar que la columna de fecha está en el formato correcto
df_pr["created_on"] = pd.to_datetime(df_pr["created_on"])

# Filtrar PRs por fecha
recent_prs = df_pr[df_pr["created_on"] >= date_limit]

# Crear una lista de todas las fechas en el rango analizado
date_range = pd.date_range(start=date_limit, end=today, freq="D")
date_range = [d.date() for d in date_range]

# Contar PRs por día
prs_by_day = recent_prs.groupby(recent_prs["created_on"].dt.date).size()

# Asegurar que tenemos todas las fechas (incluso las que no tienen PRs)
prs_by_day = prs_by_day.reindex(date_range, fill_value=0)

##### Graficar

In [ ]:
import numpy as np

# 📈 1. Gráfico general de PRs por día
plt.figure(figsize=(12, 6))
prs_by_day.plot(kind="bar", color="cornflowerblue")
plt.title(f"PRs creados por día (últimos {num_days_to_analyze} días)", fontsize=14)
plt.xlabel("Fecha", fontsize=12)
plt.ylabel("Cantidad de PRs", fontsize=12)

# Añadir etiquetas de valor sobre cada barra
for i, value in enumerate(prs_by_day):
    if value > 0:
        plt.text(i, value + 0.1, str(int(value)), ha="center")

plt.tight_layout()
plt.xticks(rotation=45)
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.show()


# 📈 2. Gráficos individuales por desarrollador
# Obtener lista de desarrolladores
developers = recent_prs["author"].unique()

for dev in developers:
    # Filtrar PRs de este desarrollador
    dev_prs = recent_prs[recent_prs["author"] == dev]

    # Contar PRs por día para este desarrollador
    dev_prs_by_day = dev_prs.groupby(dev_prs["created_on"].dt.date).size()

    # Asegurar que tenemos todas las fechas (incluso las que no tienen PRs)
    dev_prs_by_day = dev_prs_by_day.reindex(date_range, fill_value=0)

    # Crear gráfico para este desarrollador
    plt.figure(figsize=(12, 5))
    ax = dev_prs_by_day.plot(kind="bar", color="lightseagreen")
    plt.title(
        f"PRs creados por {dev} (últimos {num_days_to_analyze} días)", fontsize=14
    )
    plt.xlabel("Fecha", fontsize=12)
    plt.ylabel("Cantidad de PRs", fontsize=12)

    # Añadir etiquetas de valor sobre cada barra
    for i, value in enumerate(dev_prs_by_day):
        if value > 0:
            plt.text(i, value + 0.05, str(int(value)), ha="center")

    # Añadir información adicional
    total_prs = dev_prs_by_day.sum()
    avg_prs = dev_prs_by_day.mean()
    plt.figtext(
        0.5,
        0.01,
        f"Total: {int(total_prs)} PRs | Promedio: {avg_prs:.2f} PRs/día",
        ha="center",
        fontsize=10,
        bbox={"facecolor": "lightgray", "alpha": 0.5, "pad": 5},
    )

    plt.tight_layout()
    plt.xticks(rotation=45)
    plt.grid(axis="y", linestyle="--", alpha=0.7)
    plt.subplots_adjust(bottom=0.15)
    plt.show()


# 📈 3. Gráfico comparativo entre desarrolladores
dev_comparison = pd.DataFrame(
    {
        "Desarrollador": developers,
        "Total PRs": [
            len(recent_prs[recent_prs["author"] == dev]) for dev in developers
        ],
    }
).sort_values("Total PRs", ascending=False)

plt.figure(figsize=(10, 6))
bars = plt.bar(
    dev_comparison["Desarrollador"], dev_comparison["Total PRs"], color="teal"
)

# Añadir etiquetas de valor sobre cada barra
for bar in bars:
    height = bar.get_height()
    plt.text(
        bar.get_x() + bar.get_width() / 2.0,
        height + 0.1,
        f"{int(height)}",
        ha="center",
        fontsize=10,
    )

plt.title(
    f"Comparación de PRs creados por desarrollador (últimos {num_days_to_analyze} días)",
    fontsize=14,
)
plt.ylabel("Cantidad de PRs", fontsize=12)
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.tight_layout()
plt.show()


# Define the number of PRs to display per author
num_prs_to_display = num_days_to_analyze  # This can be changed to any value as needed

# Filtrar los últimos PRs para cada autor (basado en la variable)
author_prs = {}
for author in df_pr["author"].unique():
    # Obtener los últimos N PRs de este autor
    latest_prs = (
        df_pr[df_pr["author"] == author]
        .sort_values("created_on", ascending=False)
        .head(num_prs_to_display)
    )

    if not latest_prs.empty:
        author_prs[author] = latest_prs

# Crear una figura para cada autor
for author, prs in author_prs.items():
    # Configurar el gráfico para este autor
    fig, ax = plt.subplots(figsize=(10, 6))

    # Preparar datos para este autor
    comment_counts = []
    pr_titles = []
    actual_count = min(len(prs), num_prs_to_display)

    # Obtener datos para las barras
    for j in range(actual_count):
        comment_counts.append(prs.iloc[j]["comment_count"])
        pr_titles.append(f"{j + 1}")

    # Crear posiciones para las barras
    positions = np.arange(len(comment_counts))

    # Graficar barras para este autor
    bars = ax.bar(positions, comment_counts, color="skyblue", width=0.7)

    # Añadir etiquetas con el número de comentarios encima de cada barra
    for bar, count in zip(bars, comment_counts):
        if count > 0:
            height = bar.get_height()
            ax.text(
                bar.get_x() + bar.get_width() / 2.0,
                height + 0.3,
                str(int(count)),
                ha="center",
                fontsize=9,
            )

    # Configuración adicional del gráfico
    ax.set_title(
        f"Comentarios en los Últimos {actual_count} PRs de {author}",
        fontsize=14,
    )
    ax.set_xlabel(
        "Número de PR (del más reciente al más antiguo)",
        fontsize=12,
    )
    ax.set_ylabel("Número de Comentarios", fontsize=12)
    ax.set_xticks(positions)
    ax.set_xticklabels(pr_titles)

    # Añadir una referencia al número de ticket y estado en la parte inferior
    if actual_count > 0:
        ticket_info = []
        for j in range(actual_count):
            pr = prs.iloc[j]
            # Extraer el código del ticket (BTB-XXX) del título si existe
            title = pr["title"]
            ticket_code = ""
            import re

            match = re.search(r"\(([A-Z]+-\d+)\)", title) or re.search(
                r"\s([A-Z]+-\d+)", title
            )
            if match:
                ticket_code = match.group(1)
            else:
                # Intentar extraer cualquier código con formato BTB-XXX
                match = re.search(r"([A-Z]+-\d+)", title)
                if match:
                    ticket_code = match.group(1)

            state_info = f"[{pr['state']}]"
            ticket_info.append(f"{ticket_code} {state_info}")

        # Añadir anotaciones con información del ticket
        plt.figtext(
            0.5,
            0.01,
            "Tickets de referencia (del más reciente): "
            + " | ".join(ticket_info[:5])
            + ("..." if len(ticket_info) > 5 else ""),
            ha="center",
            fontsize=8,
            wrap=True,
        )

    ax.grid(axis="y", linestyle="--", alpha=0.7)
    plt.tight_layout(
        rect=[0, 0.05, 1, 0.95]
    )  # Dejar espacio para la anotación inferior
    plt.show()


# ----------------------------------


# 📈 Crear gráfica comparativa del promedio de comentarios por PR por autor
plt.figure(figsize=(10, 6))

# Calcular el promedio de comentarios por autor
avg_comments_by_author = {}
for author, prs in author_prs.items():
    avg_comments = prs["comment_count"].mean()
    avg_comments_by_author[author] = avg_comments

# Ordenar los autores por promedio de comentarios (descendente)
sorted_authors = sorted(
    avg_comments_by_author.items(), key=lambda x: x[1], reverse=True
)
authors = [item[0] for item in sorted_authors]
avg_values = [item[1] for item in sorted_authors]

# Crear barras
positions = np.arange(len(authors))
bars = plt.bar(positions, avg_values, color="teal", width=0.6)

# Añadir etiquetas con el valor promedio
for bar, value in zip(bars, avg_values):
    height = bar.get_height()
    plt.text(
        bar.get_x() + bar.get_width() / 2.0,
        height + 0.1,
        f"{value:.1f}",
        ha="center",
        fontsize=10,
    )

# Configurar gráfico
plt.title("Promedio de Comentarios por PR por Autor", fontsize=16)
plt.xlabel("Autor", fontsize=14)
plt.ylabel("Promedio de Comentarios", fontsize=14)
plt.xticks(positions, authors, rotation=45, ha="right")
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.tight_layout()
plt.show()

### 4. Revisión de Código

In [ ]:
%pip install seaborn

# import numpy as np
# import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt

# Analizar la calidad de revisión de código basado en los datos de PR
# Extraer datos de comentarios, aprobaciones y estados

# 1. Análisis de comentarios y revisiones por PR
comment_analysis = df_pr[
    ["author", "comment_count", "state", "repository", "days_open"]
]

# 2. Estadísticas de comentarios por estado y repo
comments_by_state = comment_analysis.groupby("state")["comment_count"].agg(
    ["mean", "median", "sum", "count"]
)
comments_by_repo = comment_analysis.groupby("repository")["comment_count"].agg(
    ["mean", "median", "sum", "count"]
)

# 3. Identificar PRs con comentarios vs sin comentarios
comment_analysis["has_comments"] = comment_analysis["comment_count"] > 0
comments_percentage = comment_analysis["has_comments"].mean() * 100

# 4. Análisis por autor (quién recibe más comentarios)
author_comments = comment_analysis.groupby("author").agg(
    {"comment_count": ["mean", "sum", "count"], "has_comments": "mean"}
)
author_comments.columns = [
    "avg_comments",
    "total_comments",
    "total_prs",
    "pct_prs_with_comments",
]
author_comments["pct_prs_with_comments"] = (
    author_comments["pct_prs_with_comments"] * 100
)

# 5. Relación entre tiempo abierto y cantidad de comentarios
correlation = comment_analysis[["days_open", "comment_count"]].corr().iloc[0, 1]

# Visualizaciones
plt.figure(figsize=(15, 12))

# 📈 1. Distribución de comentarios por PR
plt.subplot(2, 2, 1)
sns.histplot(comment_analysis["comment_count"], kde=True, bins=10)
plt.title("Distribución de Comentarios por PR")
plt.xlabel("Número de Comentarios")
plt.ylabel("Cantidad de PRs")

# 📈 2. Comentarios promedio por autor
plt.subplot(2, 2, 2)
author_order = author_comments.sort_values("avg_comments", ascending=False).index
sns.barplot(x=author_comments.loc[author_order, "avg_comments"], y=author_order)
plt.title("Comentarios Promedio por Autor")
plt.xlabel("Promedio de Comentarios")
plt.tight_layout()

# 📈 3. Porcentaje de PRs con comentarios por autor
plt.subplot(2, 2, 3)
author_order = author_comments.sort_values(
    "pct_prs_with_comments", ascending=False
).index
bars = plt.barh(
    author_order, author_comments.loc[author_order, "pct_prs_with_comments"]
)
plt.title("% de PRs con Comentarios por Autor")
plt.xlabel("Porcentaje")
# Añadir etiquetas de valor a las barras
for bar in bars:
    width = bar.get_width()
    plt.text(
        width + 1,
        bar.get_y() + bar.get_height() / 2,
        f"{width:.1f}%",
        ha="left",
        va="center",
    )
plt.xlim(0, 100)

# 📈 4. Relación entre tiempo abierto y comentarios
plt.subplot(2, 2, 4)
sns.scatterplot(x="days_open", y="comment_count", hue="author", data=comment_analysis)
plt.title(
    f"Relación entre Tiempo Abierto y Comentarios\nCorrelación: {correlation:.2f}"
)
plt.xlabel("Días Abierto")
plt.ylabel("Número de Comentarios")

plt.tight_layout()
plt.show()

# Tabla de estadísticas
print("\n=== Estadísticas de Revisión de Código ===")
print(f"PRs con al menos un comentario: {comments_percentage:.1f}%")
print(f"Comentarios promedio por PR: {comment_analysis['comment_count'].mean():.2f}")
print("\nComentarios por estado de PR:")
print(comments_by_state)
print("\nComentarios por repositorio:")
print(comments_by_repo)

# Análisis de calidad de revisión
print("\n=== Análisis de Calidad de Revisión ===")
print("Estadísticas por autor:")
print(author_comments.sort_values("avg_comments", ascending=False))

# Identificar revisores más activos (estimación basada en comentarios)
# Asumimos que autores que reciben comentarios también hacen comentarios en PRs de otros
active_reviewers = author_comments.sort_values("total_comments", ascending=False)
print("\nRevisores más activos (estimado):")
for author, row in active_reviewers.iterrows():
    print(
        f"- {author}: {row['total_comments']} comentarios totales en {row['total_prs']} PRs"
    )

# Análisis adicional de tiempo de respuesta y eficiencia
merged_prs = df_pr[df_pr["state"] == "MERGED"]
if not merged_prs.empty:
    print(f"\nTiempo promedio hasta merge: {merged_prs['days_open'].mean():.1f} días")
    print(
        f"PRs que recibieron comentarios: {merged_prs['comment_count'].gt(0).mean() * 100:.1f}%"
    )

    # Analizar si los PRs con comentarios tardan más en ser mergeados
    with_comments = merged_prs[merged_prs["comment_count"] > 0]["days_open"].mean()
    without_comments = merged_prs[merged_prs["comment_count"] == 0]["days_open"].mean()
    print(f"Tiempo promedio hasta merge (con comentarios): {with_comments:.1f} días")
    print(f"Tiempo promedio hasta merge (sin comentarios): {without_comments:.1f} días")

### Tiempo de Revisión

In [ ]:
import pandas as pd

# import numpy as np
import seaborn as sns

# Análisis de tiempo desde PR creada hasta merge/cierre
import matplotlib.pyplot as plt

# Asegurar que las fechas están en formato correcto
df_pr["created_on"] = pd.to_datetime(df_pr["created_on"])
df_pr["updated_on"] = pd.to_datetime(df_pr["updated_on"])

# Filtrar PRs que han sido cerradas (merged o declined)
closed_prs = df_pr[df_pr["state"].isin(["MERGED", "DECLINED"])].copy()

if not closed_prs.empty:
    # Calcular tiempo hasta cierre para PRs cerradas
    closed_prs["time_to_close_days"] = (
        closed_prs["updated_on"] - closed_prs["created_on"]
    ).dt.total_seconds() / 86400

    # Analizar por autor, repositorio y tipo de rama
    time_by_author = (
        closed_prs.groupby("author")["time_to_close_days"]
        .agg(["mean", "median", "count"])
        .sort_values("mean")
    )
    time_by_repo = (
        closed_prs.groupby("repository")["time_to_close_days"]
        .agg(["mean", "median", "count"])
        .sort_values("mean")
    )
    time_by_branch_type = (
        closed_prs.groupby("type_branch")["time_to_close_days"]
        .agg(["mean", "median", "count"])
        .sort_values("mean")
    )

    # Visualizaciones
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))

    # 1. Distribución general de tiempos de revisión
    sns.histplot(closed_prs["time_to_close_days"], bins=15, kde=True, ax=axes[0, 0])
    axes[0, 0].set_title("Distribución de Tiempos de Revisión")
    axes[0, 0].set_xlabel("Días hasta cierre")
    axes[0, 0].axvline(
        closed_prs["time_to_close_days"].median(),
        color="red",
        linestyle="--",
        label=f"Mediana: {closed_prs['time_to_close_days'].median():.1f} días",
    )
    axes[0, 0].axvline(
        closed_prs["time_to_close_days"].mean(),
        color="green",
        linestyle="-",
        label=f"Media: {closed_prs['time_to_close_days'].mean():.1f} días",
    )
    axes[0, 0].legend()

    # 2. Tiempo promedio por autor
    sns.barplot(x=time_by_author.index, y=time_by_author["mean"], ax=axes[0, 1])
    axes[0, 1].set_title("Tiempo Promedio de Revisión por Autor")
    axes[0, 1].set_ylabel("Días")
    axes[0, 1].set_xticklabels(axes[0, 1].get_xticklabels(), rotation=45)
    for i, v in enumerate(time_by_author["mean"]):
        axes[0, 1].text(i, v + 0.1, f"{v:.1f}", ha="center")

    # 3. Tiempo promedio por repositorio
    sns.barplot(x=time_by_repo.index, y=time_by_repo["mean"], ax=axes[1, 0])
    axes[1, 0].set_title("Tiempo Promedio de Revisión por Repositorio")
    axes[1, 0].set_ylabel("Días")
    axes[1, 0].set_xticklabels(axes[1, 0].get_xticklabels(), rotation=45)
    for i, v in enumerate(time_by_repo["mean"]):
        axes[1, 0].text(i, v + 0.1, f"{v:.1f}", ha="center")

    # 4. Boxplot de tiempo por tipo de rama
    sns.boxplot(x="type_branch", y="time_to_close_days", data=closed_prs, ax=axes[1, 1])
    axes[1, 1].set_title("Distribución de Tiempo por Tipo de Rama")
    axes[1, 1].set_ylabel("Días hasta cierre")
    axes[1, 1].set_xlabel("Tipo de rama")

    plt.tight_layout()
    plt.show()

    # Análisis adicional: Correlación entre comentarios y tiempo de revisión
    correlation = closed_prs[["time_to_close_days", "comment_count"]].corr().iloc[0, 1]

    # Visualizar relación entre comentarios y tiempo
    plt.figure(figsize=(10, 6))
    sns.scatterplot(
        x="comment_count", y="time_to_close_days", hue="author", data=closed_prs
    )
    plt.title(
        f"Relación entre Comentarios y Tiempo de Revisión (r = {correlation:.2f})"
    )
    plt.xlabel("Número de Comentarios")
    plt.ylabel("Días hasta cierre")
    plt.grid(True, linestyle="--", alpha=0.7)
    plt.show()

    # Imprimir estadísticas de tiempo de revisión
    print("===== ANÁLISIS DE TIEMPO DE REVISIÓN =====")
    print(
        f"\nTiempo promedio global hasta cierre: {closed_prs['time_to_close_days'].mean():.1f} días"
    )
    print(
        f"Tiempo mediano global hasta cierre: {closed_prs['time_to_close_days'].median():.1f} días"
    )
    print(
        f"PRs con tiempo de revisión excesivo (>7 días): {(closed_prs['time_to_close_days'] > 7).sum()} ({(closed_prs['time_to_close_days'] > 7).mean() * 100:.1f}%)"
    )

    # Identificar PRs con tiempos de revisión rápidos vs lentos
    fast_reviews = closed_prs[closed_prs["time_to_close_days"] <= 1]
    slow_reviews = closed_prs[closed_prs["time_to_close_days"] >= 7]

    print(
        f"\nPRs con revisión rápida (≤1 día): {len(fast_reviews)} ({len(fast_reviews) / len(closed_prs) * 100:.1f}%)"
    )
    print(
        f"PRs con revisión lenta (≥7 días): {len(slow_reviews)} ({len(slow_reviews) / len(closed_prs) * 100:.1f}%)"
    )

    # Analizar tendencia de tiempo de revisión a lo largo del tiempo
    closed_prs["merge_month"] = closed_prs["updated_on"].dt.strftime("%Y-%m")
    time_trend = (
        closed_prs.groupby("merge_month")["time_to_close_days"].mean().reset_index()
    )

    plt.figure(figsize=(12, 6))
    plt.plot(
        time_trend["merge_month"],
        time_trend["time_to_close_days"],
        marker="o",
        linewidth=2,
    )
    plt.title("Tendencia de Tiempo de Revisión a lo Largo del Tiempo")
    plt.xlabel("Mes")
    plt.ylabel("Tiempo Promedio de Revisión (días)")
    plt.grid(True, linestyle="--", alpha=0.7)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
else:
    print("No hay PRs cerradas para analizar")

### Actividad en Ramas

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
# from datetime import datetime, timedelta

# Análisis de Actividad en Ramas
import matplotlib.pyplot as plt

# Extraer información de ramas de los PRs
branches_info = []

# Obtener información de las ramas desde los PR
for pr in df_pr.to_dict("records"):
    # Solo procesamos si tenemos información de rama
    if "branch" in pr and "type_branch" in pr:
        # Extraer tipo de rama (feature, bugfix, etc)
        branch_type = pr["type_branch"]
        # Nombre de la rama
        branch_name = pr["branch"]
        # Repositorio
        repo = pr["repository"]
        # Fecha de creación (estimada desde la PR)
        created_date = pr["created_on"]
        # Fecha de cierre (si está cerrada)
        closed_date = pr["updated_on"] if pr["state"] != "OPEN" else None
        # Estado actual
        state = pr["state"]
        # Autor de la rama (asumimos que es el mismo que el de la PR)
        author = pr["author"]

        # Agregar a la lista
        branches_info.append(
            {
                "branch_name": branch_name,
                "branch_type": branch_type,
                "repository": repo,
                "author": author,
                "created_date": created_date,
                "closed_date": closed_date,
                "state": state,
                "days_active": (pd.Timestamp.now(tz="UTC") - created_date).days
                if state == "OPEN"
                else (closed_date - created_date).days
                if closed_date
                else None,
            }
        )

# Crear DataFrame con la información de ramas
df_branches = pd.DataFrame(branches_info)

# 1. Análisis general de ramas
branch_states = df_branches["state"].value_counts().reset_index()
branch_states.columns = ["Estado", "Cantidad"]

# 2. Análisis por tipo de rama
branch_types = df_branches["branch_type"].value_counts().reset_index()
branch_types.columns = ["Tipo", "Cantidad"]

# 3. Ramas por autor
branches_by_author = df_branches.groupby(["author", "state"]).size().unstack().fillna(0)
if "OPEN" not in branches_by_author.columns:
    branches_by_author["OPEN"] = 0
if "MERGED" not in branches_by_author.columns:
    branches_by_author["MERGED"] = 0
if "DECLINED" not in branches_by_author.columns:
    branches_by_author["DECLINED"] = 0

# 4. Tiempo activo de las ramas
branch_lifetime = (
    df_branches.dropna(subset=["days_active"])
    .groupby(["branch_type", "state"])["days_active"]
    .agg(["mean", "median", "max"])
    .reset_index()
)

# 5. Ramas abiertas vs cerradas por repositorio
branches_by_repo = (
    df_branches.groupby(["repository", "state"]).size().unstack().fillna(0)
)

# Visualizaciones
fig, axes = plt.subplots(2, 2, figsize=(16, 14))

# 1. Estado de las ramas
sns.barplot(
    x="Estado", y="Cantidad", data=branch_states, ax=axes[0, 0], palette="viridis"
)
axes[0, 0].set_title("Estado de las Ramas", fontsize=14)
for i, v in enumerate(branch_states["Cantidad"]):
    axes[0, 0].text(i, v + 0.1, str(v), ha="center")

# 2. Tipos de ramas
sns.barplot(x="Tipo", y="Cantidad", data=branch_types, ax=axes[0, 1], palette="muted")
axes[0, 1].set_title("Distribución por Tipo de Rama", fontsize=14)
for i, v in enumerate(branch_types["Cantidad"]):
    axes[0, 1].text(i, v + 0.1, str(v), ha="center")

# 3. Ramas por autor
branches_by_author.plot(kind="bar", stacked=True, ax=axes[1, 0], colormap="tab10")
axes[1, 0].set_title("Ramas por Autor y Estado", fontsize=14)
axes[1, 0].set_xlabel("Autor")
axes[1, 0].set_ylabel("Número de Ramas")
axes[1, 0].legend(title="Estado")

# 4. Tiempo promedio de las ramas abiertas
open_branches = df_branches[df_branches["state"] == "OPEN"]
if not open_branches.empty:
    sns.boxplot(x="branch_type", y="days_active", data=open_branches, ax=axes[1, 1])
    axes[1, 1].set_title("Tiempo de Ramas Abiertas por Tipo", fontsize=14)
    axes[1, 1].set_xlabel("Tipo de Rama")
    axes[1, 1].set_ylabel("Días Activa")

plt.tight_layout()
plt.show()

# 5. Gráfico adicional: Evolución de ramas a lo largo del tiempo
plt.figure(figsize=(14, 8))

# Crear fechas para el período analizado
start_date = df_branches["created_date"].min().date()
end_date = pd.Timestamp.now(tz="UTC").date()
date_range = pd.date_range(start=start_date, end=end_date, freq="D")

# Contar ramas creadas por día
df_branches["created_day"] = df_branches["created_date"].dt.date


# Imprimir estadísticas
print("\n===== ANÁLISIS DE ACTIVIDAD EN RAMAS =====")
print(f"Total de ramas analizadas: {len(df_branches)}")
print(f"Ramas por estado: {dict(branch_states.values)}")
print(f"Ramas por tipo: {dict(branch_types.values)}")
print("\nRamas abiertas por tiempo:")
if not open_branches.empty:
    open_by_time = (
        open_branches.groupby("branch_type")["days_active"]
        .agg(["count", "mean", "median", "max"])
        .reset_index()
    )
    for _, row in open_by_time.iterrows():
        print(
            f"- {row['branch_type']}: {row['count']} ramas, promedio {row['mean']:.1f} días, máximo {row['max']} días"
        )

# Detectar ramas potencialmente abandonadas (abiertas por más de 30 días)
abandoned = open_branches[open_branches["days_active"] > 30].sort_values(
    "days_active", ascending=False
)
if not abandoned.empty:
    print("\nRamas potencialmente abandonadas (>30 días abiertas):")
    for _, row in abandoned.iterrows():
        print(
            f"- {row['branch_name']} ({row['repository']}): {row['days_active']} días, autor: {row['author']}"
        )
else:
    print("\nNo hay ramas potencialmente abandonadas.")

# Analizar patrones de creación de ramas por autor
author_patterns = (
    df_branches.groupby("author")["branch_type"].value_counts().unstack().fillna(0)
)
print("\nPatrones de creación de ramas por autor:")
print(author_patterns)

### Frecuencia de Merges

In [ ]:
import pandas as pd
# import numpy as np
import seaborn as sns
# from datetime import datetime, timedelta

import matplotlib.pyplot as plt

# Filtrar solo PRs que fueron mergeados
merged_prs = df_pr[df_pr["state"] == "MERGED"].copy()

# Convertir fechas a datetime si no lo están ya
merged_prs["merge_date"] = pd.to_datetime(merged_prs["updated_on"]).dt.date

# Análisis de merges por autor
merges_by_author = merged_prs.groupby("author").size().sort_values(ascending=False)
merges_by_author.name = "total_merges"

# Análisis temporal: merges por semana por autor
merged_prs["merge_week"] = pd.to_datetime(merged_prs["updated_on"]).dt.strftime("%Y-%U")
weekly_merges = (
    merged_prs.groupby(["author", "merge_week"]).size().reset_index(name="merges")
)
avg_weekly_merges = (
    weekly_merges.groupby("author")["merges"].mean().sort_values(ascending=False)
)
avg_weekly_merges.name = "avg_weekly_merges"

# Merges por repositorio
merges_by_repo_author = (
    merged_prs.groupby(["author", "repository"]).size().unstack(fill_value=0)
)

# Crear un DataFrame de resumen
merge_stats = pd.DataFrame(
    {"Total Merges": merges_by_author, "Promedio Semanal": avg_weekly_merges}
).reset_index()
merge_stats = merge_stats.sort_values("Total Merges", ascending=False)

# Visualización: Gráfico de barras para total de merges por autor
plt.figure(figsize=(10, 6))
bars = plt.bar(
    merge_stats["author"], merge_stats["Total Merges"], color="cornflowerblue"
)

# Agregar etiquetas de valor sobre cada barra
for bar in bars:
    height = bar.get_height()
    plt.text(
        bar.get_x() + bar.get_width() / 2.0,
        height + 0.1,
        f"{int(height)}",
        ha="center",
        va="bottom",
    )

plt.title("Cantidad de Merges por Desarrollador", fontsize=15)
plt.xlabel("Desarrollador", fontsize=12)
plt.ylabel("Número de Merges", fontsize=12)
plt.xticks(rotation=45)
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.tight_layout()
plt.show()

# Visualización: Gráfico de líneas para evolución temporal de merges
plt.figure(figsize=(12, 6))

# Crear un DataFrame pivotado para visualizar merges por semana
weekly_pivot = weekly_merges.pivot(
    index="merge_week", columns="author", values="merges"
).fillna(0)

# Asegurar que tenemos todas las semanas en el rango
all_weeks = (
    pd.date_range(
        start=merged_prs["updated_on"].min().date(),
        end=merged_prs["updated_on"].max().date(),
        freq="W",
    )
    .strftime("%Y-%U")
    .tolist()
)
weekly_pivot = weekly_pivot.reindex(all_weeks, fill_value=0)

# Graficar la evolución temporal
weekly_pivot.plot(marker="o")
plt.title("Evolución de Merges por Semana por Desarrollador", fontsize=15)
plt.xlabel("Semana", fontsize=12)
plt.ylabel("Número de Merges", fontsize=12)
plt.grid(True, linestyle="--", alpha=0.7)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Visualización: Gráfico de calor para merges por repositorio
plt.figure(figsize=(11, 7))
sns.heatmap(merges_by_repo_author, annot=True, fmt="g", cmap="YlGnBu")
plt.title("Merges por Desarrollador y Repositorio", fontsize=15)
plt.ylabel("Desarrollador", fontsize=12)
plt.xlabel("Repositorio", fontsize=12)
plt.tight_layout()
plt.show()

# Estadísticas de frecuencia de merges
print("\n===== ANÁLISIS DE FRECUENCIA DE MERGES =====")
print(f"Total de PRs mergeados: {len(merged_prs)}")
print("\nFrequencia de merges por desarrollador:")
for _, row in merge_stats.iterrows():
    print(
        f"- {row['author']}: {int(row['Total Merges'])} merges totales, {row['Promedio Semanal']:.1f} merges por semana"
    )

# Calcular tiempo promedio entre merges por desarrollador
developer_merge_gaps = {}
for author in merged_prs["author"].unique():
    author_prs = merged_prs[merged_prs["author"] == author].sort_values("updated_on")
    if len(author_prs) > 1:
        gaps = []
        for i in range(1, len(author_prs)):
            gap = (
                author_prs.iloc[i]["updated_on"] - author_prs.iloc[i - 1]["updated_on"]
            ).total_seconds() / 86400  # días
            gaps.append(gap)
        developer_merge_gaps[author] = np.mean(gaps)

if developer_merge_gaps:
    print("\nTiempo promedio entre merges consecutivos:")
    for author, avg_days in sorted(developer_merge_gaps.items(), key=lambda x: x[1]):
        print(f"- {author}: {avg_days:.1f} días")